# Triangle Billiards

In [ ]:
from ipyvue_flatsurf import Widget
from flatsurf import translation_surfaces, polygons, similarity_surfaces, GL2ROrbitClosure

## Iterate over all triples (a, b, c)

In [ ]:
def ngons(vertices):
    from itertools import count
    for N in count(vertices):
        def partitions(N, M):
            if M == 1:
                yield [N]
                return
            for head in range(1, N // M + 1):
                for tail in partitions(N - head, M - 1):
                    yield [head] + tail
        
        for angles in partitions(N, vertices):
            if (sorted(angles) != angles):
                # Only consider tuples with a <= b <= c.
                continue
            if gcd(angles) != 1:
                # Only consider tuples with coprime angles.
                continue
            if any(a >= 2 * N / (vertices - 2) for a in angles):
                # Do not allow angles of 2π (or more.)
                continue
            if any(a == N / (vertices - 2) for a in angles):
                # Ignore π angles
                continue

            yield angles

triangles = ngons(vertices=3)

## Unfold the triangle (a, b, c)
Run this cell several times, to see iterate through all the triangles.

In [ ]:
(a, b, c) = next(triangles)
print(f"angles = {a,b,c}")
t = polygons.triangle(a, b, c)
B = similarity_surfaces.billiard(t)
S = B.minimal_cover('translation')
S

## Compute a Flow Decomposition
We pick a saddle connection and compute the flow decomposition in that direction.
Note that the decomposition is computed (and shown) on a Delaunay triangulation of the surface after deleting marked points.

In [ ]:
O = GL2ROrbitClosure(S.erase_marked_points())
Ds = O.decompositions(bound=64)

In [ ]:
D = next(Ds)
W = Widget(D)
W

## Pull Flow Decomposition back to Original Surface
The coloring of the cylinders might be different in the two pictures. One is computed on a Delaunay triangulation and the other is not which can lead to different ordering (and therefore coloring) of the cylinders.

In [ ]:
from flatsurf.geometry.pyflatsurf_conversion import to_pyflatsurf, to_sage_ring
deformation = to_pyflatsurf(S).eliminateMarkedPoints()
O = GL2ROrbitClosure(deformation.codomain())
direction = vector((to_sage_ring(D.vertical().vertical().x()), to_sage_ring(D.vertical().vertical().y())))
D = O.decomposition(direction, limit=64)
V = Widget(D, deformation=deformation.section())
V

## Export the Picture as an SVG
Currently, the svg is available as `.svg` once the picture has rendered in the browser.

In [ ]:
with open("billiard.svg", "wb") as svg:
  svg.write((await V.svg).encode('utf-8'))